In [1]:
import test_environment as t
from test_environment import TestEnvironment
import yaml

In [2]:
test_env = TestEnvironment()

In [3]:
with open("config.yaml", "r") as f:
    config = yaml.load(f, Loader=yaml.loader.FullLoader)

In [4]:
videos_and_labels = (test_env.get_videos_and_labels_NTHUDDD())

In [5]:
import metrics_obtention as mo
df_list = mo.create_dataset_from_videos_NTHU(videos_and_labels, target_folder="NTHUDDD_dataset/", config=config["metric_obtention"])

[]


ValueError: Length of values (0) does not match length of index (3330)